# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

Graph databases are a natural way of navigating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metadata on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions with python syntax. Submit the results by following the instructions.


### Questions

1. How many distinct countries exist in the database? [Hint: mind repetitions]
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Who is/are the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [NB: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering the number of users who reviewed it? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews of beers, taking into consideration the "overall" score they assign as a weight, how many communities are formed from these relationships? How many users are in the largest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations, or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explanations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself (please submit one Jupyter notebook per group).

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided on Moodle (apr. 6 Gb).
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `c:\Users\osavc\Documents\Nova_BDMM\_2024\Homework\Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided on Moodle (it is the same plugins as for the labs).
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `c:\Users\osavc\Documents\Nova_BDMM\_2024\Installations\Neo4JPlugins`.
- Change the code below to include your correct folder paths. Be sure that you have Docker running first:

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Users\osavc\Documents\Nova_BDMM\_2024\Installations\Neo4JPlugins":/plugins -v "C:\Users\osavc\Documents\Nova_BDMM\_2024\HW1\Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted="gds.*" --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit  before you can view the data on Neo4j webpage (let's say 3 minutes).

In [51]:
from neo4j import GraphDatabase
from pprint import pprint

In [52]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [53]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [54]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [55]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [56]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**878787**

GROUP MEMBERS:

| STUDENT NUMBER | STUDENT NAME   |
|----------------|----------------|
| 20230570       | André Lopes    |
| 20230584       | Luís Queiroz   |
| 20230596       | Raquel Mendes  |
| 20230560       | João Gonçalves |
| XXXXXX         | XXXXXX         |

#### 1. How many distinct countries exist in the database?

In [57]:
query='''
    MATCH(c:COUNTRIES)
    RETURN COUNT(DISTINCT c.name) AS DISTINCT_COUNTRIES_COUNT
'''

result= execute_read(driver, query)
pprint(result)

[<Record DISTINCT_COUNTRIES_COUNT=200>]


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

##### 2. A) Which Beer has the most reviews? 

In this scenario, when we group reviews by a specific beer ID, we find that "Breakfast Stout" (beer_id = 11757) has the highest number of reviews. However, this beer is associated with a particular brewery. To obtain a more generalized count independent of breweries, we need to group the reviews by beer name. Doing so reveals that the most reviewed beer is "IPA", totaling 31387 reviews.

In [58]:
query='''
    MATCH (b:BEERS)-[r:REVIEWED]->(re:REVIEWS)
    WITH b.name as beer_name, COUNT(*) AS review_count
    RETURN beer_name, review_count
    ORDER BY review_count DESC
    LIMIT 1
'''
result= execute_read(driver, query)
pprint(result)

[<Record beer_name='IPA' review_count=31387>]


##### 2. B) Which Brewery has the most reviews for its beers?

In this context, we're organizing breweries by their IDs, which yields the same outcome as grouping them by name because breweries sharing the same name also have identical IDs. Consequently, the most reviewed brewery is "Sierra Nevada Brewing Co.", accumulating 175161 reviews.

In [59]:
query='''
    MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:REVIEWED]->(re:REVIEWS)
    WITH br.id AS brewery_id, br.name AS brewery_name, COUNT(re) AS review_count
    RETURN brewery_id, brewery_name, review_count
    ORDER BY review_count DESC
    LIMIT 1
'''
result= execute_read(driver, query)
pprint(result)

[<Record brewery_id='140' brewery_name='Sierra Nevada Brewing Co.' review_count=175161>]


##### 2. C) Which Country has the most reviews for its beers?

Similarly, we group the countries by their names and then count the number of reviews. As a result, the country with the highest number of reviews for its beers is the "US", boasting a total of 7675804 reviews.

In [60]:
query='''
    MATCH (cn:COUNTRIES)<-[:IN]-(c:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:REVIEWED]->(re:REVIEWS)
    WITH cn.name AS country_name, COUNT(re) AS review_count
    RETURN country_name, review_count
    ORDER BY review_count DESC
    LIMIT 1
'''
result= execute_read(driver, query)
pprint(result)

[<Record country_name='US' review_count=7675804>]


#### 3. Who is/are the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

Here, we're retrieving all users who reviewed the same beers as user 2, "CTJman", and then counting the frequency of each user's appearance.

In [61]:
query='''
    MATCH (u1:USER)<-[:POSTED]-(re1:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:REVIEWED]->(re:REVIEWS)-[:POSTED]->(u2:USER)
    WHERE u1.name = 'CTJman' AND u1 <> u2
    WITH u2.name AS other_user_name, COUNT(*) AS shared_review_count
    RETURN other_user_name, shared_review_count
    ORDER BY shared_review_count DESC
    LIMIT 5
'''
result= execute_read(driver, query)
pprint(result)

[<Record other_user_name='acurtis' shared_review_count=1428>,
 <Record other_user_name='Texasfan549' shared_review_count=1257>,
 <Record other_user_name='kjkinsey' shared_review_count=1205>,
 <Record other_user_name='oline73' shared_review_count=1191>,
 <Record other_user_name='chippo33' shared_review_count=1161>]


#### 4. Which Portuguese brewery has the most beers?


Here, we're narrowing down the country nodes to those with the name "PT", then grouping them by brewery ID to count the number of beers in each brewery. Consequently, the brewery with the highest number of beers, in Portugal, is "Dois Corvos Crevejeira".

In [62]:
query='''
    MATCH (cn:COUNTRIES{name:'PT'})<-[:IN]-(c:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)
    WITH br.id AS brewery_id, br.name AS brewery_name, COUNT(b) AS stock
    RETURN brewery_id, brewery_name, stock
    ORDER BY stock DESC
    LIMIT 1
'''
result= execute_read(driver, query)
pprint(result)

[<Record brewery_id='43285' brewery_name='Dois Corvos Cervejeira' stock=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


Initially, we filter breweries by the specific ID "43285", then group them by distinct beer IDs and count the number of reviews for each beer. As a result, the most reviewed beer is "Finisterra".

In [63]:
query='''
    MATCH (br:BREWERIES{id:'43285'})-[:BREWED]->(b:BEERS)-[:REVIEWED]->(re:REVIEWS)
    WITH br.name AS brewery_name, b.id AS beer_id, b.name AS beer_name, COUNT(re) AS number_of_reviews
    RETURN brewery_name, beer_id, beer_name, number_of_reviews
    ORDER BY number_of_reviews DESC
    LIMIT 1
'''
result= execute_read(driver, query)
pprint(result)

[<Record brewery_name='Dois Corvos Cervejeira' beer_id='216709' beer_name='Finisterra' number_of_reviews=10>]


#### 6. On average how many different beer styles does each brewery produce?

Initially, we group the results by different breweries and count the number of distinct beer styles produced by each. Then, we calculate the average number of these distinct styles across all breweries. As a result, we found that, on average, a brewery produces approximately 11 different beer styles.

In [64]:
query='''
    MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH br, COUNT(DISTINCT s) AS number_of_styles
    RETURN AVG(number_of_styles) AS avg_style_count
'''
result=execute_read(driver, query)
pprint(result)

[<Record avg_style_count=10.595992516144538>]


#### 7. Which brewery produces the strongest beers according to ABV?

Here, we're grouping breweries by their names and then calculating the average ABV for the beers produced in each brewery. To ensure accuracy, NaN values are converted to null, thus preserving the integrity of the average calculation.

In [65]:
query='''
    MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
    WITH br.name AS brewery_name, AVG(coalesce(toFloat(b.abv), null)) AS avg_abv
    WHERE avg_abv IS NOT NULL
    RETURN brewery_name, avg_abv
    ORDER BY avg_abv DESC
    LIMIT 1
'''
result=execute_read(driver, query)
pprint(result)

[<Record brewery_name='1648 Brewing Company Ltd' avg_abv=25.57777777777778>]


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

For aroma we assumed it is equivalent only to the smell rating and for appearance we assumed it is equivalent only to the look rating. Our rationale behind identifying the best beer style for individuals who prioritize aroma and appearance lies in calculating the respective average ratings for smell and look. In this case we assumed it would make sense if the user did not want to leave a rating so we decided to transform the NaN values to null and then not include those for the averages. After this we just summed up the averages to see what beer style had the highest value with "New England IPA" (8.797210377972055) being the highest.

In [66]:
query='''
    MATCH (r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH s.name AS style_type, AVG(coalesce(toFloat(r.smell), null)) AS avg_smell, AVG(coalesce(toFloat(r.look), null)) AS avg_look
    WHERE avg_smell IS NOT NULL AND avg_look IS NOT NULL
    RETURN style_type, avg_smell + avg_look AS total_avg
    ORDER BY total_avg DESC
    LIMIT 2
'''
result= execute_read(driver, query)
pprint(result)

[<Record style_type='New England IPA' total_avg=8.797210377972055>,
 <Record style_type='American Imperial Stout' total_avg=8.549279798292341>]


#### 9. Using Graph Algorithms answer **one** of the following questions:
    A. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    B. Which beer is the most influential when considering the number of users who reviewed it?
    C. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users are in the largest community?

We chose to solve question B. 


__Step 1:__ Delete any previously created graph to avoid conflicts and ensure a clean slate for the new graph projection.

In [67]:
# Clear graph, graph names need to be unique

try:
    query = """
            CALL gds.graph.drop('beer_graph') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='beer_graph'>]


__Step 2:__ Create a graph to apply the PageRank algorithm. This graph consists of only two node types (users and beers). Users are the source nodes, beers are the target nodes, and each edge has a weight of 1. The relationships are represented as (:USER)-[1]->(:BEER).

In [68]:
# Creating the graph
try:
    query = """
        CALL gds.graph.project.cypher(
            'beer_graph',
            
            'MATCH (b:BEERS) RETURN id(b) AS id
             UNION
             MATCH (u:USER) RETURN id(u) AS id',
            
            'MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS) 
            RETURN id(u) AS source,1 as weight, id(b) AS target
            LIMIT 1000'
            )
      """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (b:BEERS) RETURN id(b) AS id\n             UNION\n             MATCH (u:USER) RETURN id(u) AS id' relationshipQuery='MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS) \n            RETURN id(u) AS source,1 as weight, id(b) AS target\n            LIMIT 1000' graphName='beer_graph' nodeCount=882681 relationshipCount=1000 projectMillis=932>]


__Step 3:__ After creating the graph, we can apply the PageRank algorithm to discover which beer is the most influential. However, it's important to note that due to limiting the relations to 1000, the accuracy of the results may be compromised to expedite the computation process. We ran the code without a limit, and "Heady Topper" emerged as the most influential beer, with a corresponding score of 193.42. However, given the limit we applied, we would feel comfortable identifying any of the top 20 beers in the output as the most influential.

In [69]:
# Running the PageRank algorithm

try:
    query = """
        CALL gds.pageRank.stream('beer_graph', {relationshipWeightProperty:'weight'})
            
            YIELD nodeId, score
            
            RETURN nodeId,gds.util.asNode(nodeId).name AS name, score
            
            ORDER BY score DESC
            LIMIT 20
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeId=239820 name='Witte Wheat Ale' score=0.15012750000000002>,
 <Record nodeId=240574 name='Cherry Chocolate' score=0.15012750000000002>,
 <Record nodeId=239451 name='Blue Sunday Sour' score=0.15012750000000002>,
 <Record nodeId=239809 name='Small Craft Warning' score=0.15012750000000002>,
 <Record nodeId=239965 name='Maharaja' score=0.15012750000000002>,
 <Record nodeId=240245 name='Salted Caramel' score=0.15012750000000002>,
 <Record nodeId=238393 name='Winter Warmer' score=0.15012750000000002>,
 <Record nodeId=238733 name='Hootspa Wheat' score=0.15012750000000002>,
 <Record nodeId=239641 name='New Grist Gluten-Free Pilsner Style Beer' score=0.15012750000000002>,
 <Record nodeId=239686 name='Mango Magnifico' score=0.15012750000000002>,
 <Record nodeId=239911 name='Tricerahops Double IPA' score=0.15012750000000002>,
 <Record nodeId=239912 name='Happily Ebony After' score=0.15012750000000002>,
 <Record nodeId=240004 name="Veldensteiner Zwick'l" score=0.15012750000000002>,
 <

#### 10. Using Graph Algorithms answer **one** of the following questions:
    A. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    B. Which user is the most influential when it comes to reviews made?

 We chose to solve question B.

__Step 1:__ Delete any previously created graph to avoid conflicts and ensure a clean slate for the new graph projection.

In [70]:
# Step 0 - Clear graph

try:
    query = """CALL gds.graph.drop('user_review_graph') YIELD graphName;  """
    result = execute_read(driver, query)
    pprint(result)

except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `user_review_graph` does not exist on database `neo4j`. It might exist on another database.')


__Step 2:__ Create a graph to apply the PageRank algorithm. This graph consists of only two node types (users and reviews). The relationships are represented as (:REVIEWS)-[1]->(:USERS).

In [71]:
# Creating the graph
try:
    query = """
        CALL gds.graph.project.cypher(
            'user_review_graph',
            
            "MATCH (n) 
            WHERE head(Labels(n))='USER' or head(Labels(n))='REVIEWS' 
            RETURN id(n) AS id",
            
            'MATCH (r:REVIEWS)-[:POSTED]->(u:USER)
            RETURN id(r) AS source,1 as weight, id(u) AS target'
            )
      """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery="MATCH (n) \n            WHERE head(Labels(n))='USER' or head(Labels(n))='REVIEWS' \n            RETURN id(n) AS id" relationshipQuery='MATCH (r:REVIEWS)-[:POSTED]->(u:USER)\n            RETURN id(r) AS source,1 as weight, id(u) AS target' graphName='user_review_graph' nodeCount=9278044 relationshipCount=9073109 projectMillis=30382>]


__Step 3:__ After creating the graph, we can apply the PageRank algorithm to discover which user is the most influential.

In [72]:
# Running the PageRank algorithm

try:
    query = """
        CALL gds.pageRank.stream('user_review_graph', {relationshipWeightProperty:'weight'})
            
            YIELD nodeId, score
            
            RETURN nodeId,gds.util.asNode(nodeId).name AS name, score
            
            ORDER BY score DESC
            LIMIT 1
        """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeId=9494470 name='Sammy' score=1759.2675000004458>]


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

### As avid IMDB users we appreciate its rating system, therefore we have concocted a series of queries in order to try and evaluate which of these beers would that sort system classify as the best ones. As not everyone shares the same ideas for what constitutes a good beer or even appreciates it, it seems this might be a good way to recommend beers, as perhaps a means of popular vote rather than personal biases would be more fitting.
### We also recognize that a beer suggested should still be available and produced, because of that we assume that a beer must not be retired, meaning {retired:'f'}, in order to be considered
According to the IMDB's Wikipedia page the following formula is used to calculate the Top 250 Films' list:
$$
W = \frac{R \times v + C \times m}{v + m}
$$
##### where:
- $W$ is the weighted rating;
- $R$ is the mean rating for the movie, from 1 to 10;
- $v$ is the number of votes for the movie;
- $m$ is the minimum votes required to be listed in the Top 250;
- $C$ is the mean vote across the whole report;

##### We have then determined that these variables will take the following shape for our scenario:
- $W$ is the weighted score;
- $R$ is the average score of a given beer;
- $v$ is the number of reviews of said beer;
- $m$ is the minimum number of reviews required to be considered one of the candidates for Best Beer;
- $C$ is the global average review score;

#### Since we have to set a criteria to choose $m$, for this case, we considered that the minimum number of reviews required to be considered one of the candidates for Best Beer must be higher than the average number of votes for the 250 most reviewed beers.

In [73]:
# query='''
#     MATCH (b:BEERS)-[:REVIEWED]->(re:REVIEWS)
#     WITH b, COUNT(re) AS review_count
#     ORDER BY review_count DESC
#     LIMIT 250
#     WITH AVG(review_count) AS average_review_count
#     RETURN average_review_count
# '''
# result= execute_read(driver, query)
# pprint(result)

[<Record average_review_count=5994.220000000005>]


### $m$ = 6000

### With the goal in mind already set, and the requirements needed to proceed to get the Top 3 beers, in accordance to what an IMDB rating system would be in this scenario, the following query outputs the id, the name and the weighted score of those beers.

In [74]:
query ='''
    CALL {
        MATCH (re:REVIEWS)
        RETURN AVG(coalesce(toFloat(re.score), null)) AS global_review_avg
    }
    MATCH (beer:BEERS{retired:'f'})-[:REVIEWED]->(review:REVIEWS)
    WITH beer.id AS beer_id, 
     beer.name AS beer_name,
     AVG(coalesce(toFloat(review.score), null)) AS avg_score,
     COUNT(review) AS review_count,
     global_review_avg
    RETURN beer_id, 
       beer_name, 
       ((avg_score * review_count + global_review_avg * 6000) / (review_count + 6000)) AS weighted_score
    ORDER BY weighted_score DESC
    LIMIT 3
'''
result = execute_read(driver, query)
pprint(result)

[<Record beer_id='16814' beer_name='Heady Topper' weighted_score=4.470239725267033>,
 <Record beer_id='7971' beer_name='Pliny The Elder' weighted_score=4.428548387626803>,
 <Record beer_id='19960' beer_name='KBS (Kentucky Breakfast Stout)' weighted_score=4.3811816309485385>]


### Making sure that this weighted score does indeed level review scores based on the number of reviews a beer has.

In [75]:
# query ='''
#     MATCH (beer:BEERS)-[:REVIEWED]->(review:REVIEWS)
#     WHERE beer.id='16814' OR beer.id='7971' or beer.id='19960'
#     RETURN DISTINCT beer.name AS beer_name, AVG(coalesce(toFloat(review.score), null)) AS avg_score, COUNT(review) AS review_count
# '''
# result = execute_read(driver, query)
# pprint(result)

[<Record beer_name='Heady Topper' avg_score=4.713771880688968 review_count=14282>,
 <Record beer_name='Pliny The Elder' avg_score=4.6447993568701005 review_count=14927>,
 <Record beer_name='KBS (Kentucky Breakfast Stout)' avg_score=4.602003600630115 review_count=13331>]


In [76]:
# query ='''
#     MATCH (beer:BEERS)-[:REVIEWED]->(review:REVIEWS)
#     WITH beer.name AS beer_name, AVG(coalesce(toFloat(review.score), null)) AS avg_score, COUNT(review) AS review_count
#     WHERE review_count > 100
#     RETURN beer_name, avg_score, review_count
#     ORDER BY avg_score DESC
#     LIMIT 3
# '''
# result = execute_read(driver, query)
# pprint(result)

[<Record beer_name='Kentucky Brunch Brand Stout' avg_score=4.83773504273504 review_count=702>,
 <Record beer_name='Rare Bourbon County Brand Stout (2010)' avg_score=4.800911722141827 review_count=1382>,
 <Record beer_name='Good Morning' avg_score=4.780885466794998 review_count=1039>]
